In [1]:
import os
import mlflow
from dotenv import load_dotenv
from catboost import CatBoostRegressor
import numpy as np

logged_model = 'runs:/016a8bed9c27468c869d774ea10d768b/models'

mlflow.set_tracking_uri('http://127.0.0.1:5000')

TRACKING_SERVER_HOST = "0.0.0.0"#"127.0.0.1"
TRACKING_SERVER_PORT = 5000

In [2]:
# подгружаем .env
load_dotenv()

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" # ваш код здесь
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv('AWS_SECRET_ACCESS_KEY')

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [3]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2025/08/14 16:41:29 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.6.0, required: scikit-learn==1.3.1)
 - catboost (current: 1.2.7, required: catboost==1.2.2)
 - nbformat (current: 5.10.4, required: nbformat==5.10.3)
 - boto3 (current: 1.34.77, required: boto3==1.34.78)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [4]:
type(loaded_model)

mlflow.pyfunc.PyFuncModel

In [5]:
mlflow.pyfunc.get_default_pip_requirements()

['cloudpickle==2.2.1']

In [6]:
import pandas as pd
data = pd.read_csv('/home/mle-user/mle_projects/mle-project-sprint-3-v001/data/data_Polyn_Kbins_OnrHot_AutoFeat.csv')

In [8]:
data['build_type_floors_low_rise'] = 0.0

In [9]:
df = data

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df.head()

,log_price,floor,is_apartment,kitchen_area_old,living_area_old,rooms,studio,total_area_old,build_year,building_type_int,ceiling_height,flats_count,floors_total,has_elevator,build_type_floors,age,floor_ratio,kitchen_ratio,living_ratio,building_price_mean,building_area_mean,location_cluster,dist_origin_dest,distance_to_metro_fast,ceiling_height_cluster,flats_count_cluster,floor_group,build_height_group,bad_floor_flag,total_area,kitchen_area,living_area,num_fg__Polynomial__1,num_fg__Polynomial__floor,num_fg__Polynomial__rooms,num_fg__Polynomial__building_type_int,num_fg__Polynomial__ceiling_height,num_fg__Polynomial__dist_origin_dest,num_fg__Polynomial__distance_to_metro_fast,num_fg__Polynomial__total_area,num_fg__Polynomial__kitchen_area,num_fg__Polynomial__living_area,num_fg__Polynomial__floor^2,num_fg__Polynomial__floor rooms,num_fg__Polynomial__floor building_type_int,num_fg__Polynomial__floor ceiling_height,num_fg__Polynomial__floor dist_origin_dest,num_fg__Polynomial__floor distance_to_metro_fast,num_fg__Polynomial__floor total_area,num_fg__Polynomial__floor kitchen_area,num_fg__Polynomial__floor living_area,num_fg__Polynomial__rooms^2,num_fg__Polynomial__rooms building_type_int,num_fg__Polynomial__rooms ceiling_height,num_fg__Polynomial__rooms dist_origin_dest,num_fg__Polynomial__rooms distance_to_metro_fast,num_fg__Polynomial__rooms total_area,num_fg__Polynomial__rooms kitchen_area,num_fg__Polynomial__rooms living_area,num_fg__Polynomial__building_type_int^2,num_fg__Polynomial__building_type_int ceiling_height,num_fg__Polynomial__building_type_int dist_origin_dest,num_fg__Polynomial__building_type_int distance_to_metro_fast,num_fg__Polynomial__building_type_int total_area,num_fg__Polynomial__building_type_int kitchen_area,num_fg__Polynomial__building_type_int living_area,num_fg__Polynomial__ceiling_height^2,num_fg__Polynomial__ceiling_height dist_origin_dest,num_fg__Polynomial__ceiling_height distance_to_metro_fast,num_fg__Polynomial__ceiling_height total_area,num_fg__Polynomial__ceiling_height kitchen_area,num_fg__Polynomial__ceiling_height living_area,num_fg__Polynomial__dist_origin_dest^2,num_fg__Polynomial__dist_origin_dest distance_to_metro_fast,num_fg__Polynomial__dist_origin_dest total_area,num_fg__Polynomial__dist_origin_dest kitchen_area,num_fg__Polynomial__dist_origin_dest living_area,num_fg__Polynomial__distance_to_metro_fast^2,num_fg__Polynomial__distance_to_metro_fast total_area,num_fg__Polynomial__distance_to_metro_fast kitchen_area,num_fg__Polynomial__distance_to_metro_fast living_area,num_fg__Polynomial__total_area^2,num_fg__Polynomial__total_area kitchen_area,num_fg__Polynomial__total_area living_area,num_fg__Polynomial__kitchen_area^2,num_fg__Polynomial__kitchen_area living_area,num_fg__Polynomial__living_area^2,num_fg__Polynomial__floor^3,num_fg__Polynomial__floor^2 rooms,num_fg__Polynomial__floor^2 building_type_int,num_fg__Polynomial__floor^2 ceiling_height,num_fg__Polynomial__floor^2 dist_origin_dest,num_fg__Polynomial__floor^2 distance_to_metro_fast,num_fg__Polynomial__floor^2 total_area,num_fg__Polynomial__floor^2 kitchen_area,num_fg__Polynomial__floor^2 living_area,num_fg__Polynomial__floor rooms^2,num_fg__Polynomial__floor rooms building_type_int,num_fg__Polynomial__floor rooms ceiling_height,num_fg__Polynomial__floor rooms dist_origin_dest,num_fg__Polynomial__floor rooms distance_to_metro_fast,num_fg__Polynomial__floor rooms total_area,num_fg__Polynomial__floor rooms kitchen_area,num_fg__Polynomial__floor rooms living_area,num_fg__Polynomial__floor building_type_int^2,num_fg__Polynomial__floor building_type_int ceiling_height,num_fg__Polynomial__floor building_type_int dist_origin_dest,num_fg__Polynomial__floor building_type_int distance_to_metro_fast,num_fg__Polynomial__floor building_type_int total_area,num_fg__Polynomial__floor building_type_int kitchen_area,num_fg__Polynomial__floor building_type_int living_area,num_fg__Polynomial__floor ceiling_height^2,num_fg__Polynomial__floor ceiling_hei

In [12]:
data[["build_type_floors_low_rise",
                "building_area_mean",
                "building_price_mean",
                "ceiling_height.1",
                "dist_origin_dest",
                "living_area.1",
                "num_fg__KBinsDiscretizer__kitchen_area",
                "num_fg__Polynomial__building_type_int kitchen_area^2",
                "num_fg__Polynomial__ceiling_height total_area",
                "num_fg__Polynomial__ceiling_height total_area kitchen_area",
                "num_fg__Polynomial__ceiling_height total_area^2",
                "num_fg__Polynomial__dist_origin_dest",
                "num_fg__Polynomial__dist_origin_dest distance_to_metro_fast^2",
                "num_fg__Polynomial__dist_origin_dest^2 distance_to_metro_fast",
                "num_fg__Polynomial__dist_origin_dest^2 kitchen_area",
                "num_fg__Polynomial__dist_origin_dest^2 living_area",
                "num_fg__Polynomial__dist_origin_dest^2 total_area",
                "num_fg__Polynomial__floor building_type_int distance_to_metro_fast",
                "num_fg__Polynomial__floor building_type_int^2",
                "num_fg__Polynomial__floor ceiling_height total_area",
                "num_fg__Polynomial__floor total_area^2",
                "num_fg__Polynomial__living_area^2",
                "num_fg__Polynomial__rooms^2",
                "num_fg__Polynomial__total_area living_area^2",
                "num_fg__Polynomial__total_area^2 kitchen_area",
                "num_fg__Polynomial__total_area^2 living_area",
                "total_area.1",
                "total_area_old"]].iloc[[1]]

,build_type_floors_low_rise,building_area_mean,building_price_mean,ceiling_height.1,dist_origin_dest,living_area.1,num_fg__KBinsDiscretizer__kitchen_area,num_fg__Polynomial__building_type_int kitchen_area^2,num_fg__Polynomial__ceiling_height total_area,num_fg__Polynomial__ceiling_height total_area kitchen_area,num_fg__Polynomial__ceiling_height total_area^2,num_fg__Polynomial__dist_origin_dest,num_fg__Polynomial__dist_origin_dest distance_to_metro_fast^2,num_fg__Polynomial__dist_origin_dest^2 distance_to_metro_fast,num_fg__Polynomial__dist_origin_dest^2 kitchen_area,num_fg__Polynomial__dist_origin_dest^2 living_area,num_fg__Polynomial__dist_origin_dest^2 total_area,num_fg__Polynomial__floor building_type_int distance_to_metro_fast,num_fg__Polynomial__floor building_type_int^2,num_fg__Polynomial__floor ceiling_height total_area,num_fg__Polynomial__floor total_area^2,num_fg__Polynomial__living_area^2,num_fg__Polynomial__rooms^2,num_fg__Polynomial__total_area living_area^2,num_fg__Polynomial__total_area^2 kitchen_area,num_fg__Polynomial__total_area^2 living_area,total_area.1,total_area_old
1,0.0,55.966667,1.873333e+07,3.0,4.772297,26.4,1.0,551.120025,129.0,2141.400049,5547.0,4.772297,1.318621e+06,11971.571317,378.061964,601.25515,979.317114,7359.093221,28.0,903.0,12943.0,696.95998,1.0,29969.279134,30693.400705,48813.599295,43.0,43.0


In [23]:
y_pred = loaded_model.predict(data[["build_type_floors_low_rise",
                "building_area_mean",
                "building_price_mean",
                "ceiling_height.1",
                "dist_origin_dest",
                "living_area.1",
                "num_fg__KBinsDiscretizer__kitchen_area",
                "num_fg__Polynomial__building_type_int kitchen_area^2",
                "num_fg__Polynomial__ceiling_height total_area",
                "num_fg__Polynomial__ceiling_height total_area kitchen_area",
                "num_fg__Polynomial__ceiling_height total_area^2",
                "num_fg__Polynomial__dist_origin_dest",
                "num_fg__Polynomial__dist_origin_dest distance_to_metro_fast^2",
                "num_fg__Polynomial__dist_origin_dest^2 distance_to_metro_fast",
                "num_fg__Polynomial__dist_origin_dest^2 kitchen_area",
                "num_fg__Polynomial__dist_origin_dest^2 living_area",
                "num_fg__Polynomial__dist_origin_dest^2 total_area",
                "num_fg__Polynomial__floor building_type_int distance_to_metro_fast",
                "num_fg__Polynomial__floor building_type_int^2",
                "num_fg__Polynomial__floor ceiling_height total_area",
                "num_fg__Polynomial__floor total_area^2",
                "num_fg__Polynomial__living_area^2",
                "num_fg__Polynomial__rooms^2",
                "num_fg__Polynomial__total_area living_area^2",
                "num_fg__Polynomial__total_area^2 kitchen_area",
                "num_fg__Polynomial__total_area^2 living_area",
                "total_area.1",
                "total_area_old"]].iloc[[1]])

In [24]:
y_pred = np.expm1(y_pred*4)
y_pred

array([11396487.49213262])

In [25]:
np.expm1(data['log_price'].iloc[[1]])

1    13500000.0
Name: log_price, dtype: float64

In [26]:
# импорт библиотек
from sklearn.metrics import (mean_absolute_error, 
                             mean_squared_error, 
                             mean_absolute_percentage_error,
                             mean_squared_log_error,
                             median_absolute_error, 
                             r2_score,
                             root_mean_squared_error
                             )
from permetrics.regression import RegressionMetric


In [29]:
df.iloc[[1]].shape

(1, 291)

In [28]:
y_test_real = np.expm1(data['log_price'].iloc[[1]])
y_pred_real = y_pred
metrics_final = {}

evaluator = RegressionMetric(y_test_real.to_numpy(), y_pred_real)

def adjusted_r2_score(y_true, y_pred, x_shape):
    n, k = x_shape
    ssr = np.sum((y_true - y_pred) ** 2)
    sst = np.sum((y_true - y_true.mean()) ** 2)
    r2 = 1 - ssr / sst
    return 1 - (1 - r2) * (n - 1) / (n - k - 1)

def wape_score(y_true, y_pred):
    numerator = np.sum(np.abs(y_true - y_pred))
    denominator = np.sum(np.abs(y_true))

    return numerator / denominator

rmspe = mean_squared_percentage_error(y_test_real, y_pred, square_root=True) *100
mae = mean_absolute_error(y_test_real, y_pred_real) 
mse = mean_squared_error(y_test_real, y_pred_real)  
rmse = root_mean_squared_error(y_test_real, y_pred_real)  
mspe = mean_squared_percentage_error(y_test_real, y_pred_real, square_root=False)
rmspe = mean_squared_percentage_error(y_test_real, y_pred_real, square_root=True) 
medae = median_absolute_error(y_test_real, y_pred_real)
r2 = r2_score(y_test_real, y_pred_real)             
r2_adj = adjusted_r2_score(y_test_real, y_pred_real, df.iloc[[1]].shape) 
mape = mean_absolute_percentage_error(y_test_real, y_pred_real) * 100 
smape = mean_absolute_percentage_error(y_test_real, y_pred_real)    
msle = mean_squared_log_error(y_test_real, y_pred_real) 
rmsle = np.sqrt(msle)  
wape = wape_score(y_test_real, y_pred_real) 

metrics_final = {
    "MAE": mae,
    "MSE": mse,
    "RMSE": rmse,
    "MSPE": mspe,
    "RMSPE": rmspe,
    "MedAE": medae,
    "MAPE_proc": mape,
    "SMAPE": smape,
    "MSLE": msle,
    "RMSLE": rmsle,
    "R_2": r2,
    "Adjusted R_2": r2_adj,
    "WAPE": wape
}
metrics_final

/home/mle-user/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/tmp/ipykernel_4273/1131434111.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  r2 = 1 - ssr / sst
/tmp/ipykernel_4273/1131434111.py:12: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (1 - r2) * (n - 1) / (n - k - 1)


{'MAE': 2103512.5078673735,
 'MSE': 4424764870754.487,
 'RMSE': 2103512.5078673735,
 'MSPE': 0.02427854524419474,
 'RMSPE': 0.15581574132350923,
 'MedAE': 2103512.5078673735,
 'MAPE_proc': 15.581574132350923,
 'SMAPE': 0.15581574132350923,
 'MSLE': 0.02869110158177833,
 'RMSLE': 0.16938447857397776,
 'R_2': nan,
 'Adjusted R_2': nan,
 'WAPE': 0.15581574132350923}

In [18]:
data['log_price'].describe()

count    123937.000000
mean         16.401600
std           0.678020
min           9.210440
25%          16.001562
50%          16.283610
75%          16.630919
max          23.013144
Name: log_price, dtype: float64